# Clustering Crypto

In [1]:
!pip install -U altair

     |████████████████████████████████| 727 kB 4.7 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
# import hvplot.pandas
# import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [3]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [3]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [5]:
# Alternatively, use the provided csv file:

from pathlib import Path
df = pd.read_csv("crypto_data.csv")
df.set_index('Unnamed: 0', inplace=True)
df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [6]:
# Keep only necessary columns:

cols = ['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']
df = df[cols]
df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [7]:
# Keep only cryptocurrencies that are trading
df_trading = df[df.IsTrading == True]
df_trading.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [8]:
# Keep only cryptocurrencies with a working algorithm

sum(df_trading.Algorithm.isnull())
# All cryptos have a working algorithm

0

In [9]:
# Remove the "IsTrading" column

df_trading = df_trading.drop(columns = ['IsTrading'])

In [10]:
df_trading.head(3)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000


In [11]:
# Remove rows with at least 1 null value
for column in df_trading.columns:
    print(f"Column {column} has {df_trading[column].isnull().sum()} null values")
print(f"Duplicate entries: {df_trading.duplicated().sum()}")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values
Duplicate entries: 0


In [12]:
df_trading.dropna(inplace=True)

In [13]:
for column in df_trading.columns:
    print(f"Column {column} has {df_trading[column].isnull().sum()} null values")
print(f"Duplicate entries: {df_trading.duplicated().sum()}")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 0 null values
Column TotalCoinSupply has 0 null values
Duplicate entries: 0


In [14]:
df_trading.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [15]:
# Remove rows with cryptocurrencies having no coins mined

df_trading = df_trading[df_trading.TotalCoinsMined != 0]
df_trading.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [16]:
# Drop rows where there are 'N/A' text values
df_trading.isnull().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [17]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df

coins_name = df_trading.CoinName.to_frame()
coins_name.head()

,CoinName
Unnamed: 0,
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [18]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm

df_trading.drop(columns=['CoinName'], inplace=True)
df_trading.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [19]:
df_trading.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [20]:
# Create dummy variables for text features

X = pd.get_dummies(df_trading, columns=["Algorithm", "ProofType"])
X.shape
X.head(3)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [21]:
# Standardize data

data_scaler = StandardScaler()
data_scaler.fit(X)

StandardScaler()

In [22]:
X_scaled = data_scaler.transform(X)
X_scaled[1:]

array([[-0.09358885, -0.14499604, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [ 0.52587231,  4.4937636 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11635442, -0.15255408, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       ...,
       [-0.09523411, -0.13215444, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11658774, -0.15255408, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11674507, -0.15284989, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ]])

### Reducing Dimensions Using PCA

In [23]:
# Use PCA to reduce dimensions to 3 principal components

pca = PCA(n_components=3)
coins_pca = pca.fit_transform(X_scaled)

In [24]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(
    data=coins_pca, columns=["PC 1", "PC 2", "PC 3"], index=df_trading.index
)
pcs_df.head()

,PC 1,PC 2,PC 3
Unnamed: 0,,,
42,-0.363032,1.016211,-0.636471
404,-0.346365,1.016322,-0.636974
1337,2.300602,1.477366,-0.685932
BTC,-0.137153,-1.380689,0.236432
ETH,-0.141408,-1.988696,0.398950


In [25]:
pca.explained_variance_ratio_

array([0.02736718, 0.02093058, 0.02009902])

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [26]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)


elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow

,k,inertia
0,1,3645.548872
1,2,2477.361873
2,3,1490.472017
3,4,585.906568
4,5,417.035003
5,6,293.331549
6,7,227.215379
7,8,179.198972
8,9,148.142762
9,10,124.239953


In [27]:
# Create the Elbow Curve using AltAir

import altair as alt
alt.Chart(df_elbow).mark_circle(size=50).encode(
    x='k',
    y='inertia'
).interactive()


alt.Chart(...)

Running K-Means with `k=4`

In [29]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

In [30]:
# Create a new DataFrame named clustered_df, that includes the following columns 
# "Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply", "PC 1", "PC 2", "PC 3", "CoinName", "Class". 
# You should maintain the index of the crypto_df

# Create a new DataFrame including predicted clusters and cryptocurrencies features

clustered_data = {"Algorithm": df_trading.Algorithm,
               "ProofType": df_trading.ProofType, 
               "TotalCoinsMined": df_trading.TotalCoinsMined, 
               "TotalCoinSupply": df_trading.TotalCoinSupply,
               "PC 1": pcs_df["PC 1"], 
               "PC 2": pcs_df["PC 2"],
               "PC 3": pcs_df["PC 3"], 
               "CoinName": coins_name.CoinName,
               "Class": model.labels_
              }
clustered_df = pd.DataFrame(data = clustered_data, index=df_trading.index, )

clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
Unnamed: 0,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.363032,1.016211,-0.636471,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.346365,1.016322,-0.636974,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.300602,1.477366,-0.685932,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.137153,-1.380689,0.236432,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.141408,-1.988696,0.398950,Ethereum,1


### Visualizing Results: Altair

#### 3D-Scatter with Clusters

In [33]:
# Create a 2D-Scatter with the PCA data and the clusters

alt.Chart(clustered_df).mark_circle(size=60).encode(
    x='PC 1',
    y='PC 2',
    color='Class',
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
).interactive()


alt.Chart(...)

In [35]:
# Scale TotalCoinsMined & TotalCoinSuppy by dividing 1MM
clustered_df_scaled = clustered_df
clustered_df_scaled["TotalCoinsMined"] = clustered_df_scaled["TotalCoinsMined"] / 1000000
clustered_df_scaled = clustered_df_scaled.astype({'TotalCoinSupply': 'float64'})
clustered_df_scaled.dtypes
clustered_df_scaled["TotalCoinsMined"] = clustered_df_scaled["TotalCoinsMined"] / 1000000
clustered_df_scaled.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
Unnamed: 0,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e-11,4.200000e+01,-0.363032,1.016211,-0.636471,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e-03,5.320000e+08,-0.346365,1.016322,-0.636974,404Coin,0
1337,X13,PoW/PoS,2.927942e-02,3.141593e+11,2.300602,1.477366,-0.685932,EliteCoin,0
BTC,SHA-256,PoW,1.792717e-05,2.100000e+07,-0.137153,-1.380689,0.236432,Bitcoin,1
ETH,Ethash,PoW,1.076842e-04,0.000000e+00,-0.141408,-1.988696,0.398950,Ethereum,1


In [36]:
alt.Chart(clustered_df_scaled).mark_circle(size=60).encode(
    x='TotalCoinsMined',
    y='TotalCoinSupply',
    color='Class',
).interactive()


alt.Chart(...)